In [1]:
import os 

In [2]:
%pwd

'c:\\Users\\asus\\OneDrive\\Desktop\\Pandas app\\End-to-end-ML-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\asus\\OneDrive\\Desktop\\Pandas app\\End-to-end-ML-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    status_file: str
    unzip_data_dir: Path
    all_schema: dict

In [6]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml,create_directories

In [28]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_Validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema=self.schema.COLUMNS
        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            status_file=config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            all_schema=schema
        )

        return data_validation_config

In [29]:
import os
from mlProject import logger
import pandas as pd

In [52]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config=config
    def validate_all_cols(self) -> bool:
        try:
            validation_status = True
            messages = []

            # Read the data
            data = pd.read_csv(self.config.unzip_data_dir)
            column_data_types = data.dtypes.apply(lambda x: str(x)).to_dict()
            all_cols = list(column_data_types.keys())
            data_dtypes = list(column_data_types.values())
            all_schema = list(self.config.all_schema.keys())
            all_schema_dtype = dict(self.config.all_schema)

            # Check for missing columns
            missing_cols = [col for col in all_cols if col not in all_schema]
            if missing_cols:
                validation_status = False
                messages.append(f"Insufficient Keys: Missing columns {missing_cols}")

            # Check for data type mismatches
            mismatched_keys = {key: (column_data_types[key], all_schema_dtype[key]) for key in column_data_types if key in all_schema_dtype and column_data_types[key] != all_schema_dtype[key]}
            if mismatched_keys:
                validation_status = False
                messages.append(f"Type Mismatch: {mismatched_keys}")

            # Write status to file
            with open(self.config.status_file, 'w') as file:
                file.write(f"Validation status is: {validation_status}\n")
                for message in messages:
                    file.write(f"{message}\n")

            return validation_status

        except Exception as e:
            raise e


In [53]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_Validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_cols()
except Exception as e:
    raise e

2024-09-06 19:23:30,706 : INFO : common : yaml file: config\config.yaml loaded successfully
2024-09-06 19:23:30,706 : INFO : common : yaml file: config\config.yaml loaded successfully
2024-09-06 19:23:30,708 : INFO : common : yaml file: params.yaml loaded successfully
2024-09-06 19:23:30,708 : INFO : common : yaml file: params.yaml loaded successfully
2024-09-06 19:23:30,712 : INFO : common : yaml file: schema.yaml loaded successfully
2024-09-06 19:23:30,712 : INFO : common : yaml file: schema.yaml loaded successfully
2024-09-06 19:23:30,714 : INFO : common : created directory at: artifacts
2024-09-06 19:23:30,714 : INFO : common : created directory at: artifacts
2024-09-06 19:23:30,716 : INFO : common : created directory at: artifacts/data_validation
2024-09-06 19:23:30,716 : INFO : common : created directory at: artifacts/data_validation
